# VAR model (for model evaluation)
* The VAR model is trained on the train set. The forecasted independent variables of the test set will be incorporated in the Multiple Linear Regression model to predict China's HRC prices.

Sources for the development of the VAR model
* https://github.com/fickaz/time-series-for-business/blob/master/Forecasting.ipynb
* https://gist.github.com/BioSciEconomist/197bd86ea61e0b4a49707af74a0b9f9c

In [1]:
# Import libraries
import numpy as np 
import pandas as pd 

from statsmodels.tsa.api import VAR
from statsmodels.stats.stattools import durbin_watson

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Read csv
file_path = '../data/final/wo_na.csv'
df = pd.read_csv(file_path)

df.set_index('Date', inplace=True)
df.index = pd.to_datetime(df.index)

In [3]:
# After feature selection, these are the shortlisted variables
list_of_variables = ['Iron Ore (CFR, $/t)', 'HCC (Aus FOB, $/t)',
         'Domestic Scrap (DDP Jiangsu incl. VAT $/t)',
         'Monthly Export of Semis & Finished Steel as % of Production',
         'FAI in urban real estate development (y-o-y) Growth',
         'Automobile Production (y-o-y)', 'Civil Metal-Vessels/Steel Ships (y-o-y)',
         'Household Fridges (y-o-y)', 'Air Conditioner (y-o-y)']
hrc = ['HRC (FOB, $/t)']
final_cols = hrc + list_of_variables

final_df = df.copy()
final_df = final_df[final_cols]
final_df.head()

,"HRC (FOB, $/t)","Iron Ore (CFR, $/t)","HCC (Aus FOB, $/t)",Domestic Scrap (DDP Jiangsu incl. VAT $/t),Monthly Export of Semis & Finished Steel as % of Production,FAI in urban real estate development (y-o-y) Growth,Automobile Production (y-o-y),Civil Metal-Vessels/Steel Ships (y-o-y),Household Fridges (y-o-y),Air Conditioner (y-o-y)
Date,,,,,,,,,,
2006-09-01,472,59,116,252,14.277778,24.3,24.0,6.0,34.2,3.9
2006-10-01,477,62,85,260,13.657895,24.1,25.1,11.4,31.8,6.7
2006-11-01,470,62,84,262,16.078947,24.0,26.1,8.3,28.9,7.2
2006-12-01,470,61,90,270,15.512821,21.8,27.0,13.1,15.8,16.1
2007-01-01,470,62,98,273,13.026316,24.3,45.0,8.6,15.8,16.1


In [4]:
# Split dataset into train and test set
final_df_train, final_df_test = final_df[0:-17], final_df[-17:]

print(final_df_train.shape)
print(final_df_test.shape)

(201, 10)
(17, 10)


In [5]:
# Difference data to achieve stationarity (as tested in 01_stationarity_tests, stationarity was achieved for the train set after 1 round of differencing)
final_df_train_differenced = final_df_train.diff().dropna()

In [6]:
# Determine the best number of lags
model = VAR(final_df_train_differenced)
x = model.select_order(maxlags=12)
x.summary()

,AIC,BIC,FPE,HQIC
0,45.13,45.30*,3.965e+19,45.20*
1,44.92,46.82,3.245e+19*,45.69
2,45.02,48.64,3.611e+19,46.49
3,45.19,50.52,4.334e+19,47.35
4,45.32,52.38,5.157e+19,48.18
5,45.52,54.29,6.721e+19,49.07
6,45.64,56.14,8.449e+19,49.89
7,45.97,58.20,1.372e+20,50.93
8,46.11,60.05,1.930e+20,51.76
9,46.22,61.89,2.864e+20,52.57


In [7]:
# Fit model with optimal lag
model_fitted = model.fit(4)
model_fitted.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Mon, 21, Apr, 2025
Time:                     01:59:48
--------------------------------------------------------------------
No. of Equations:         10.0000    BIC:                    51.9450
Nobs:                     196.000    HQIC:                   47.8639
Log likelihood:          -6789.72    FPE:                4.06056e+19
AIC:                      45.0878    Det(Omega_mle):     6.07664e+18
--------------------------------------------------------------------
Results for equation HRC (FOB, $/t)
                                                                    coefficient       std. error           t-stat            prob
---------------------------------------------------------------------------------------------------------------------------------
const                                                                  0.597111         1.930510            0.309  

In [8]:
# Employ the Durbin Watson's Statistic to check for serial correlation in the residuals.
# As values are close to 2, there is no significant serial correlation.

out = durbin_watson(model_fitted.resid)

for col, val in zip(final_df.columns, out):
    print(col, ':', round(val, 2))

HRC (FOB, $/t) : 1.99
Iron Ore (CFR, $/t) : 2.03
HCC (Aus FOB, $/t) : 2.04
Domestic Scrap (DDP Jiangsu incl. VAT $/t) : 2.05
Monthly Export of Semis & Finished Steel as % of Production : 2.0
FAI in urban real estate development (y-o-y) Growth : 1.94
Automobile Production (y-o-y) : 1.92
Civil Metal-Vessels/Steel Ships (y-o-y) : 2.0
Household Fridges (y-o-y) : 1.99
Air Conditioner (y-o-y) : 1.98


In [9]:
# Using the last 4 observations (since lag order is 4) to forecast the following periods
lag_order = model_fitted.k_ar
forecast_input = final_df_train_differenced.values[-lag_order:]
forecast_input

array([[ 25.        ,   3.        ,  51.        ,   3.        ,
         -4.57136237,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [ 19.        ,   1.        , -24.        ,  -1.        ,
         -1.89663462,  -0.1       ,   8.9       ,  14.        ,
          0.5       ,   1.8       ],
       [-43.        , -11.        , -76.        , -25.        ,
          0.30813172,  -0.4       ,   9.4       ,   8.4       ,
          2.6       ,  -0.4       ],
       [-62.        , -11.        , -38.        , -37.        ,
          0.76200717,  -1.        ,   2.8       ,  -5.        ,
          1.4       ,   2.        ]])

In [10]:
# Forecast the following periods
fc = model_fitted.forecast(y=forecast_input, steps=17)
fc_period = pd.date_range(start=final_df_test.index[0], periods=17, freq='MS')
df_forecast = pd.DataFrame(fc, index=fc_period, columns=final_df_test.columns + '_1d')
df_forecast.index.name = 'Date'
df_forecast

,"HRC (FOB, $/t)_1d","Iron Ore (CFR, $/t)_1d","HCC (Aus FOB, $/t)_1d",Domestic Scrap (DDP Jiangsu incl. VAT $/t)_1d,Monthly Export of Semis & Finished Steel as % of Production_1d,FAI in urban real estate development (y-o-y) Growth_1d,Automobile Production (y-o-y)_1d,Civil Metal-Vessels/Steel Ships (y-o-y)_1d,Household Fridges (y-o-y)_1d,Air Conditioner (y-o-y)_1d
Date,,,,,,,,,,
2023-06-01,-32.490591,-2.762687,-31.132718,-9.653816,0.540204,-2.535823,-8.706377,0.682229,-4.565753,-3.373138
2023-07-01,-15.321327,-4.087397,-2.801269,-17.730032,-0.908565,-0.465823,1.417069,-2.306445,-2.285393,-2.028395
2023-08-01,4.283680,1.548765,8.057582,-0.305713,-0.641916,0.890272,3.719912,-2.020236,1.816967,-0.541384
2023-09-01,-3.180402,1.239149,7.618220,5.854620,-0.301716,-0.528608,-0.438996,1.167968,1.282562,0.545343
2023-10-01,1.221296,-0.674855,5.124143,5.392257,0.181939,-0.365406,-0.447329,0.476020,-0.160005,0.551397
2023-11-01,9.803027,2.540354,-2.680294,7.401498,-0.016141,-0.981590,-0.498302,0.224326,0.250460,-0.420459
2023-12-01,5.645224,2.138176,2.221151,-0.121359,-0.004278,-0.195812,0.513879,0.146023,0.420449,0.066395
2024-01-01,0.364980,1.504756,0.158260,-0.291031,0.002069,0.207612,-0.466222,1.127856,0.165522,0.135995
2024-02-01,-2.280558,-0.482777,0.940492,-0.146800,-0.031877,-0.008891,0.152965,-0.351998,-0.333995,0.317966


In [11]:
# Invert differencing of forecasted results
def invert_transformation(df_train, df_forecast):
    df_fc = df_forecast.copy()
    cols = df_train.columns
    for col in cols:        
        df_fc[str(col)+'_forecast'] = df_train[col].iloc[-1] + df_fc[str(col)+'_1d'].cumsum()
    return df_fc

df_forecast_processed = invert_transformation(final_df_train, df_forecast)

df_forecast_processed.to_csv('../data/final/var_testset.csv', index=True)

# VAR model (for actual forecast)
* The VAR model is trained on the full dataset.

In [12]:
# Read csv
file_path = '../data/final/wo_na.csv'
df2 = pd.read_csv(file_path)

df2.set_index('Date', inplace=True)
df2.index = pd.to_datetime(df2.index)

In [13]:
# Filter for shortlisted variables
list_of_variables = ['Iron Ore (CFR, $/t)', 'HCC (Aus FOB, $/t)',
         'Domestic Scrap (DDP Jiangsu incl. VAT $/t)',
         'Monthly Export of Semis & Finished Steel as % of Production',
         'FAI in urban real estate development (y-o-y) Growth',
         'Automobile Production (y-o-y)', 'Civil Metal-Vessels/Steel Ships (y-o-y)',
         'Household Fridges (y-o-y)', 'Air Conditioner (y-o-y)']
hrc = ['HRC (FOB, $/t)']
final_cols = hrc + list_of_variables

final_df2 = df2.copy()
final_df2 = final_df2[final_cols]
final_df2.head()

,"HRC (FOB, $/t)","Iron Ore (CFR, $/t)","HCC (Aus FOB, $/t)",Domestic Scrap (DDP Jiangsu incl. VAT $/t),Monthly Export of Semis & Finished Steel as % of Production,FAI in urban real estate development (y-o-y) Growth,Automobile Production (y-o-y),Civil Metal-Vessels/Steel Ships (y-o-y),Household Fridges (y-o-y),Air Conditioner (y-o-y)
Date,,,,,,,,,,
2006-09-01,472,59,116,252,14.277778,24.3,24.0,6.0,34.2,3.9
2006-10-01,477,62,85,260,13.657895,24.1,25.1,11.4,31.8,6.7
2006-11-01,470,62,84,262,16.078947,24.0,26.1,8.3,28.9,7.2
2006-12-01,470,61,90,270,15.512821,21.8,27.0,13.1,15.8,16.1
2007-01-01,470,62,98,273,13.026316,24.3,45.0,8.6,15.8,16.1


In [14]:
# Difference data to achieve stationarity (as tested in 01_stationarity_tests, stationarity was achieved for the full dataset after 1 round of differencing)
final_df2_differenced = final_df2.diff().dropna()

In [15]:
# Determine the best number of lags
model2 = VAR(final_df2_differenced)
x2 = model2.select_order(maxlags=12)
x2.summary()

,AIC,BIC,FPE,HQIC
0,44.88,45.04*,3.098e+19,44.95*
1,44.62,46.40,2.387e+19*,45.34
2,44.72,48.12,2.660e+19,46.10
3,44.90,49.92,3.229e+19,46.93
4,45.00,51.65,3.704e+19,47.69
5,45.22,53.49,4.843e+19,48.56
6,45.33,55.22,5.875e+19,49.33
7,45.74,57.25,9.849e+19,50.39
8,46.01,59.14,1.506e+20,51.32
9,46.24,60.99,2.339e+20,52.20


In [16]:
# Fit model with optimal lag
model_fitted2 = model2.fit(4)
model_fitted2.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Mon, 21, Apr, 2025
Time:                     01:59:48
--------------------------------------------------------------------
No. of Equations:         10.0000    BIC:                    51.2743
Nobs:                     213.000    HQIC:                   47.4190
Log likelihood:          -7383.98    FPE:                3.01529e+19
AIC:                      44.8042    Det(Omega_mle):     5.18546e+18
--------------------------------------------------------------------
Results for equation HRC (FOB, $/t)
                                                                    coefficient       std. error           t-stat            prob
---------------------------------------------------------------------------------------------------------------------------------
const                                                                  0.585135         1.847010            0.317  

In [17]:
# Employ the Durbin Watson's Statistic to check for serial correlation in the residuals.
# As values are close to 2, there is no significant serial correlation.

out2 = durbin_watson(model_fitted2.resid)

for col, val in zip(final_df2.columns, out2):
    print(col, ':', round(val, 2))

HRC (FOB, $/t) : 1.97
Iron Ore (CFR, $/t) : 2.02
HCC (Aus FOB, $/t) : 2.04
Domestic Scrap (DDP Jiangsu incl. VAT $/t) : 2.03
Monthly Export of Semis & Finished Steel as % of Production : 2.02
FAI in urban real estate development (y-o-y) Growth : 1.94
Automobile Production (y-o-y) : 1.95
Civil Metal-Vessels/Steel Ships (y-o-y) : 2.0
Household Fridges (y-o-y) : 1.99
Air Conditioner (y-o-y) : 1.98


In [18]:
# Using the last 4 observations (since lag order is 4) to forecast the following periods
lag_order2 = model_fitted2.k_ar
forecast_input2 = final_df2_differenced.values[-lag_order2:]
forecast_input2

array([[-20.        ,  -1.        , -12.        ,  -2.        ,
         -0.07713463,  -0.1       ,  -1.2       ,   0.1       ,
         -2.4       ,  -4.3       ],
       [-34.        ,  -7.        , -31.        , -30.        ,
          2.74575224,   0.        ,  -1.1       ,  -0.3       ,
          0.        ,  -1.7       ],
       [-11.        ,  -5.        , -19.        ,  -9.        ,
          1.002331  ,   0.1       ,  -0.7       ,   0.        ,
          0.2       ,   0.2       ],
       [ 44.        ,  10.        ,  17.        ,  17.        ,
          0.45232816,  -0.2       ,   0.3       ,   0.2       ,
          1.        ,   0.2       ]])

In [19]:
# Forecast the following periods
fc2 = model_fitted2.forecast(y=forecast_input2, steps=17)
fc_period2 = pd.date_range(start='11/1/2024', periods=17, freq='MS')
df_forecast2 = pd.DataFrame(fc2, index=fc_period2, columns=final_df2.columns + '_1d')
df_forecast2.index.name = 'Date'
df_forecast2

,"HRC (FOB, $/t)_1d","Iron Ore (CFR, $/t)_1d","HCC (Aus FOB, $/t)_1d",Domestic Scrap (DDP Jiangsu incl. VAT $/t)_1d,Monthly Export of Semis & Finished Steel as % of Production_1d,FAI in urban real estate development (y-o-y) Growth_1d,Automobile Production (y-o-y)_1d,Civil Metal-Vessels/Steel Ships (y-o-y)_1d,Household Fridges (y-o-y)_1d,Air Conditioner (y-o-y)_1d
Date,,,,,,,,,,
2024-11-01,19.405733,2.146433,28.112562,-3.217672,-0.228613,2.331490,6.274981,0.159794,3.465096,3.158250
2024-12-01,0.393028,0.441906,16.970414,1.988942,-0.320143,1.021310,3.126926,2.299420,1.253820,1.830514
2025-01-01,5.100750,1.941511,-5.196327,9.813901,0.267119,-1.137923,-2.308399,-0.453018,-2.027208,-0.742831
2025-02-01,10.632432,2.049517,-8.549116,2.248489,0.336962,-0.304782,-1.762744,-1.263097,-0.881281,-0.304889
2025-03-01,6.852293,3.026626,0.196417,2.328163,0.174999,0.047624,0.321265,0.359870,0.814319,0.705993
2025-04-01,-2.246912,-0.442230,2.933537,-2.730486,-0.006803,0.165588,0.068102,0.719057,0.185949,0.663057
2025-05-01,-4.174677,-2.045662,1.972330,-3.576975,-0.185422,0.172952,0.016791,0.427123,-0.405187,-0.094933
2025-06-01,-0.795270,-1.488854,-0.494445,0.136381,-0.074824,-0.388813,-0.723971,-0.856438,-0.586360,-0.376777
2025-07-01,0.310747,0.053009,-1.673525,0.924078,0.049814,-0.364902,-0.975668,-0.502533,-0.325724,-0.343127


In [20]:
# Invert differencing of forecasted results
df_forecast2_processed = invert_transformation(final_df2, df_forecast2)

df_forecast2_processed.to_csv('../data/final/var_forecast_actual.csv', index=True)